# Swaps

In [ ]:
#| default_exp swap

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from sugar.quote import Quote
from sugar.helpers import apply_slippage
from sugar.pool import LiquidityPoolForSwap
from enum import IntEnum
from web3 import Web3
from typing import List, Dict, Any, Union, TypedDict, Optional
from decimal import Decimal
import copy
from web3 import Web3


In [ ]:
#| export

class CommandType(IntEnum):
    V3_SWAP_EXACT_IN = 0x00
    V3_SWAP_EXACT_OUT = 0x01
    SWEEP = 0x04
    V2_SWAP_EXACT_IN = 0x08
    V2_SWAP_EXACT_OUT = 0x09
    WRAP_ETH = 0x0b
    UNWRAP_WETH = 0x0c

# Define ABI types for each command
ABI_DEFINITION = {
    CommandType.V3_SWAP_EXACT_IN: [
        {"type": "address"},
        {"type": "uint256"},
        {"type": "uint256"},
        {"type": "bytes"},
        {"type": "bool"}
    ],
    CommandType.V2_SWAP_EXACT_IN: [
        {"type": "address"},
        {"type": "uint256"},
        {"type": "uint256"},
        {"type": "tuple[]", "components": [
            {"type": "address", "name": "from"},
            {"type": "address", "name": "to"},
            {"type": "bool", "name": "stable"}
        ]},
        {"type": "bool"}
    ],
    CommandType.V2_SWAP_EXACT_OUT: [
        {"type": "address"},
        {"type": "uint256"},
        {"type": "uint256"},
        {"type": "tuple[]", "components": [
            {"type": "address", "name": "from"},
            {"type": "address", "name": "to"},
            {"type": "bool", "name": "stable"}
        ]},
        {"type": "bool"}
    ],
    CommandType.V3_SWAP_EXACT_OUT: [
        {"type": "address"},
        {"type": "uint256"},
        {"type": "uint256"},
        {"type": "bytes"},
        {"type": "bool"}
    ],
    CommandType.WRAP_ETH: [
        {"type": "address"},
        {"type": "uint256"}
    ],
    CommandType.UNWRAP_WETH: [
        {"type": "address"},
        {"type": "uint256"}
    ],
    CommandType.SWEEP: [
        {"type": "address"},
        {"type": "address"},
        {"type": "uint256"}
    ]
}

class RoutePlanner:
    def __init__(self):
        """Initialize a new RoutePlanner"""
        self.commands = "0x"
        self.inputs: List[str] = []
        self.web3 = Web3()

    def add_command(self, command_type: CommandType, parameters: List[Any]) -> None:
        """
        Add a command to the route planner
        
        Args:
            command_type: Type of command to add
            parameters: Parameters for the command
        """
        # Get the ABI definition for this command
        abi_types = ABI_DEFINITION[command_type]
        
        # Handle tuple[] encoding for V2 swap commands
        if command_type in [CommandType.V2_SWAP_EXACT_IN, CommandType.V2_SWAP_EXACT_OUT]:
            # For paths parameter (index 3), we need to reformat for web3.py encoding
            paths = parameters[3]
            # No need for custom encoding since web3.py handles tuples properly
        
        # Encode the parameters according to the ABI
        encoded_input = self.web3.codec.encode_abi(abi_types, parameters)
        encoded_hex = encoded_input.hex()
        
        # Add to inputs
        self.inputs.append(encoded_hex)
        
        # Add command byte to commands
        command_hex = format(command_type, '02x')
        self.commands = self.commands + command_hex

    def get_encoded_commands(self) -> str:
        """Get the encoded commands"""
        return self.commands
    
    def get_encoded_inputs(self) -> List[str]:
        """Get the encoded inputs"""
        return self.inputs

In [ ]:
#| export

# Constants
CONTRACT_BALANCE_FOR_V3_SWAPS = int("0x8000000000000000000000000000000000000000000000000000000000000000", 16)

def pack_route(nodes: List[LiquidityPoolForSwap]) -> bytes:
    """Pack route elements into bytes format for the router"""
    # This is a placeholder - actual implementation would depend on how routes are packed
    # For now, we'll just return a simple placeholder bytes object
    return b'0x' + b''.join(bytes.fromhex(node['lp'][2:]) for node in nodes)

def setup_planner(
    quote: Quote,
    slippage: float,
    account: Optional[str] = None,
    route_planner: Optional[RoutePlanner] = None
) -> RoutePlanner:
    """Setup route planner with the given quote and chain"""
    
    if route_planner is None: route_planner = RoutePlanner()
    
    router_address = "addrrrr" # TODO: <<<< add router address here 
    min_amount_out = apply_slippage(quote.amount_out, slippage)
    
    # By default money comes from contract
    tokens_come_from_contract = False
    
    # Handle wrapped native token if needed
    if quote.from_token.wrapped_token_address:
        # When trading from native token, wrap token first
        route_planner.add_command(CommandType.WRAP_ETH, [router_address, quote.amount_in])
        tokens_come_from_contract = True
    
    # Group nodes by pool type (v2 or v3)
    grouped_nodes: List[List[LiquidityPoolForSwap]] = []
    
    for node in quote.path:
        if not grouped_nodes: grouped_nodes.append([node])
        elif node.type < 1:
            # Current node is a v2 pool
            if float(grouped_nodes[-1][0].type) < 1: grouped_nodes[-1].append(node)
            else: grouped_nodes.append([node])
        else:
            # Current node is a v3 pool
            if grouped_nodes[-1][0].type >= 1: grouped_nodes[-1].append(node)
            else: grouped_nodes.append([node])
    
    if len(grouped_nodes) == 1:
        # All nodes belong to the same pool type
        nodes = grouped_nodes[0]
        is_v2_pool = float(nodes[0]['type']) < 1
        
        route_planner.add_command(
            CommandType.V2_SWAP_EXACT_IN if is_v2_pool else CommandType.V3_SWAP_EXACT_IN,
            [
                # Where should money go?
                router_address if quote['toToken'].get('wrappedAddress') else account,
                quote['amount'],
                min_amount_out,
                [
                    {
                        'from': n['from_'], 
                        'to': n['to'], 
                        'stable': float(n['type']) == 0
                    } for n in nodes
                ] if is_v2_pool else pack_route(nodes),
                not tokens_come_from_contract,
            ]
        )
    #else:
        # Mixed v2 and v3 pools
        # first_batch = grouped_nodes[0]
        # last_batch = grouped_nodes[-1]
        # rest = grouped_nodes[1:-1]
        
        # # Handle first batch
        # is_first_batch_v2 = float(first_batch[0]['type']) < 1
        # next_batch = rest[0] if rest else last_batch
        
        # route_planner.add_command(
        #     CommandType.V2_SWAP_EXACT_IN if is_first_batch_v2 else CommandType.V3_SWAP_EXACT_IN,
        #     [
        #         router_address if is_first_batch_v2 else next_batch[0]['lp'],
        #         quote['amount'],
        #         0,  # No expectations on min amount out for first batch
        #         [
        #             {
        #                 'from': n['from_'], 
        #                 'to': n['to'], 
        #                 'stable': float(n['type']) == 0
        #             } for n in first_batch
        #         ] if is_first_batch_v2 else pack_route(first_batch),
        #         not tokens_come_from_contract,
        #     ]
        # )
        
        # # Handle middle batches
        # for idx, batch in enumerate(rest):
        #     is_batch_v2 = float(batch[0]['type']) < 1
        #     next_batch = rest[idx + 1] if idx + 1 < len(rest) else last_batch
            
        #     route_planner.add_command(
        #         CommandType.V2_SWAP_EXACT_IN if is_batch_v2 else CommandType.V3_SWAP_EXACT_IN,
        #         [
        #             router_address if is_batch_v2 else next_batch[0]['lp'],
        #             0 if is_batch_v2 else CONTRACT_BALANCE_FOR_V3_SWAPS,
        #             0,  # No expectations for middle batches
        #             [
        #                 {
        #                     'from': n['from_'], 
        #                     'to': n['to'], 
        #                     'stable': float(n['type']) == 0
        #                 } for n in batch
        #             ] if is_batch_v2 else pack_route(batch),
        #             False,  # Money comes from contract
        #         ]
        #     )
        
        # # Handle last batch
        # is_last_batch_v2 = float(last_batch[0]['type']) < 1
        
        # route_planner.add_command(
        #     CommandType.V2_SWAP_EXACT_IN if is_last_batch_v2 else CommandType.V3_SWAP_EXACT_IN,
        #     [
        #         router_address if quote['toToken'].get('wrappedAddress') else account,
        #         0 if is_last_batch_v2 else CONTRACT_BALANCE_FOR_V3_SWAPS,
        #         min_amount_out,
        #         [
        #             {
        #                 'from': n['from_'], 
        #                 'to': n['to'], 
        #                 'stable': float(n['type']) == 0
        #             } for n in last_batch
        #         ] if is_last_batch_v2 else pack_route(last_batch),
        #         False,  # Money comes from contract
        #     ]
        # )
    
    # Handle unwrapping WETH if needed
    if quote.to_token.wrapped_token_address: route_planner.add_command(CommandType.UNWRAP_WETH, [account, min_amount_out])
    
    return route_planner

NameError: name 'RouteElement' is not defined

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()